In [1]:
import numpy as np
import pandas as pd
import os

In [2]:
ship_movements = pd.read_csv(r'C:\Users\User\Downloads\vessel_movements_PPT.csv')

In [3]:
ship_movements

,imo,mmsi,vessel_id,vessel_name,date_of_build,vessel_type,group,timestamp,date_only,lon,...,anchorage,terminal,maneuvering_zone,p,vref,sfc_me,sfc_ae,sfc_ab,ael,abl
0,9223253,525119038,692017,PRIMA TANGGUH LVI,2002-04-01,Crude/Oil Products Tanker,Crude & Products Tanker,2023-09-17T00:01:42.000Z,2023-09-17,103.826065,...,PPT Anchorage,NaN,NaN,7860.0,14.5,185.00,230.0,300.0,507.0,365.0
1,9223253,525119038,692017,PRIMA TANGGUH LVI,2002-04-01,Crude/Oil Products Tanker,Crude & Products Tanker,2023-09-17T00:01:44.000Z,2023-09-17,103.826065,...,PPT Anchorage,NaN,NaN,7860.0,14.5,185.00,230.0,300.0,507.0,365.0
2,9223253,525119038,692017,PRIMA TANGGUH LVI,2002-04-01,Crude/Oil Products Tanker,Crude & Products Tanker,2023-09-17T00:07:43.000Z,2023-09-17,103.825690,...,PPT Anchorage,NaN,NaN,7860.0,14.5,185.00,230.0,300.0,507.0,365.0
3,9223253,525119038,692017,PRIMA TANGGUH LVI,2002-04-01,Crude/Oil Products Tanker,Crude & Products Tanker,2023-09-17T00:07:45.000Z,2023-09-17,103.825690,...,PPT Anchorage,NaN,NaN,7860.0,14.5,185.00,230.0,300.0,507.0,365.0
4,9223253,525119038,692017,PRIMA TANGGUH LVI,2002-04-01,Crude/Oil Products Tanker,Crude & Products Tanker,2023-09-17T00:10:42.000Z,2023-09-17,103.825580,...,PPT Anchorage,NaN,NaN,7860.0,14.5,185.00,230.0,300.0,507.0,365.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2822594,9708693,374766000,714547,MSC CLARA,2015-11-01,Container Ship (Fully Cellular),Container Ship,2023-08-21T21:59:17.000Z,2023-08-21,103.761665,...,NaN,NaN,PPT Maneouvering Zone,62500.0,19.0,175.00,220.0,300.0,NaN,NaN
2822595,9708693,374766000,714547,MSC CLARA,2015-11-01,Container Ship (Fully Cellular),Container Ship,2023-08-21T22:02:17.000Z,2023-08-21,103.761665,...,NaN,NaN,PPT Maneouvering Zone,62500.0,19.0,175.00,220.0,300.0,4493.0,908.0
2822596,9449106,636020361,647347,MENDELSSOHN,2012-04-01,Container Ship (Fully Cellular),Container Ship,2023-08-21T21:58:47.000Z,2023-08-21,103.783330,...,NaN,NaN,PPT Maneouvering Zone,31990.0,23.0,174.83,220.0,300.0,NaN,NaN
2822597,9449106,636020361,647347,MENDELSSOHN,2012-04-01,Container Ship (Fully Cellular),Container Ship,2023-08-21T22:01:47.000Z,2023-08-21,103.783330,...,NaN,NaN,PPT Maneouvering Zone,31990.0,23.0,174.83,220.0,300.0,2503.0,436.0


In [4]:
import geopandas as gpd
from shapely.geometry import Point

maneuvering_zone = gpd.read_file(r'C:\Users\User\Downloads\drive-download-20240126T095513Z-001\PPT_maneuveringzone.shp')
port_boundary = gpd.read_file(r'C:\Users\User\Downloads\drive-download-20240126T095513Z-001\PPT_portboundary.shp')
berth_polygon = gpd.read_file(r'C:\Users\User\Downloads\drive-download-20240126T095513Z-001\PPT_berths.shp')
anchorage = gpd.read_file(r'C:\Users\User\Downloads\drive-download-20240126T095513Z-001\PPT_anchorage.shp')

In [5]:
# Function to determine if a point is outside maneuvering zone but inside port fence
def is_outside_maneuvering_zone_inside_port_fence(lon, lat):
    point = Point(lon, lat)
    return point.within(port_boundary.unary_union) and not point.within(maneuvering_zone.unary_union)

def is_inside_maneuvering_zone(lon, lat):
    point = Point(lon, lat)
    return point.within(maneuvering_zone.unary_union)

# Function to determine if a point is within the berth polygon
def is_within_berth(lon, lat):
    point = Point(lon, lat)
    return point.within(berth_polygon.unary_union)

def is_within_anchorage(lon, lat):
    point = Point(lon, lat)
    return point.within(anchorage.unary_union)

def determine_operating_mode(row):
    moving_statuses = {0, 3, 4, 8, 11, 12}
    speed_condition = row['speed'] > 1
    status_condition = row['nav_stat'] in moving_statuses
    location_condition = is_outside_maneuvering_zone_inside_port_fence(row['lon'], row['lat'])
    
    if speed_condition or status_condition:
        if location_condition:
            return "Transit"
        elif is_inside_maneuvering_zone(row['lon'], row['lat']):
            return "Maneuvering"
        else: 
            return "Unknown"
    elif not speed_condition:
        if is_within_berth(row['lon'], row['lat']):
            return "Hotel"
        elif is_within_anchorage(row['lon'], row['lat']):
            return "Anchorage"
        else :
            return "Unknown"
    else:
        return "Unknown"

# Apply the function to create the 'operating_mode' column
ship_movements['operating_mode'] = ship_movements.apply(determine_operating_mode, axis=1)


In [6]:
ship_movements

,imo,mmsi,vessel_id,vessel_name,date_of_build,vessel_type,group,timestamp,date_only,lon,...,terminal,maneuvering_zone,p,vref,sfc_me,sfc_ae,sfc_ab,ael,abl,operating_mode
0,9223253,525119038,692017,PRIMA TANGGUH LVI,2002-04-01,Crude/Oil Products Tanker,Crude & Products Tanker,2023-09-17T00:01:42.000Z,2023-09-17,103.826065,...,NaN,NaN,7860.0,14.5,185.00,230.0,300.0,507.0,365.0,Anchorage
1,9223253,525119038,692017,PRIMA TANGGUH LVI,2002-04-01,Crude/Oil Products Tanker,Crude & Products Tanker,2023-09-17T00:01:44.000Z,2023-09-17,103.826065,...,NaN,NaN,7860.0,14.5,185.00,230.0,300.0,507.0,365.0,Anchorage
2,9223253,525119038,692017,PRIMA TANGGUH LVI,2002-04-01,Crude/Oil Products Tanker,Crude & Products Tanker,2023-09-17T00:07:43.000Z,2023-09-17,103.825690,...,NaN,NaN,7860.0,14.5,185.00,230.0,300.0,507.0,365.0,Anchorage
3,9223253,525119038,692017,PRIMA TANGGUH LVI,2002-04-01,Crude/Oil Products Tanker,Crude & Products Tanker,2023-09-17T00:07:45.000Z,2023-09-17,103.825690,...,NaN,NaN,7860.0,14.5,185.00,230.0,300.0,507.0,365.0,Anchorage
4,9223253,525119038,692017,PRIMA TANGGUH LVI,2002-04-01,Crude/Oil Products Tanker,Crude & Products Tanker,2023-09-17T00:10:42.000Z,2023-09-17,103.825580,...,NaN,NaN,7860.0,14.5,185.00,230.0,300.0,507.0,365.0,Anchorage
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2822594,9708693,374766000,714547,MSC CLARA,2015-11-01,Container Ship (Fully Cellular),Container Ship,2023-08-21T21:59:17.000Z,2023-08-21,103.761665,...,NaN,PPT Maneouvering Zone,62500.0,19.0,175.00,220.0,300.0,NaN,NaN,Maneuvering
2822595,9708693,374766000,714547,MSC CLARA,2015-11-01,Container Ship (Fully Cellular),Container Ship,2023-08-21T22:02:17.000Z,2023-08-21,103.761665,...,NaN,PPT Maneouvering Zone,62500.0,19.0,175.00,220.0,300.0,4493.0,908.0,Maneuvering
2822596,9449106,636020361,647347,MENDELSSOHN,2012-04-01,Container Ship (Fully Cellular),Container Ship,2023-08-21T21:58:47.000Z,2023-08-21,103.783330,...,NaN,PPT Maneouvering Zone,31990.0,23.0,174.83,220.0,300.0,NaN,NaN,Maneuvering
2822597,9449106,636020361,647347,MENDELSSOHN,2012-04-01,Container Ship (Fully Cellular),Container Ship,2023-08-21T22:01:47.000Z,2023-08-21,103.783330,...,NaN,PPT Maneouvering Zone,31990.0,23.0,174.83,220.0,300.0,2503.0,436.0,Maneuvering


In [7]:
#step 2:
def calculate_event_duration(df):
    df.sort_values(by=['imo', 'timestamp'], inplace=True)
    
    # Initialize a new column 'time_diff' with NaN
    df['time_diff'] = np.nan
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    
    prev_imo = None
    prev_timestamp = None
    current_event_duration = 0
    
    for index, row in df.iterrows():
        current_imo = row['imo']
        current_timestamp = row['timestamp']
        
        # Check if IMO is the same as the previous row
        if current_imo == prev_imo:
            # Calculate time difference
            time_diff = (current_timestamp - prev_timestamp).total_seconds()
            
            # Check if the time difference is greater than 10,000 seconds
            if time_diff > 10000:
                current_event_duration = 0  # Reset event duration if not continuous
            else:
                current_event_duration += time_diff  # Add time difference to event duration
        
        # Update 'Event_Duration' column
        df.at[index, 'time_diff'] = current_event_duration
        
        # Update previous values for the next iteration
        prev_imo = current_imo
        prev_timestamp = current_timestamp
    
    return df

calculate_event_duration(ship_movements)
ship_movements

,imo,mmsi,vessel_id,vessel_name,date_of_build,vessel_type,group,timestamp,date_only,lon,...,maneuvering_zone,p,vref,sfc_me,sfc_ae,sfc_ab,ael,abl,operating_mode,time_diff
2663186,1006946,319646000,660707,PARAFFIN,2001-10-01,Yacht,Other,2023-09-30 02:20:00+00:00,2023-09-30,103.783820,...,PPT Maneouvering Zone,2984.0,14.0,195.0,230.0,300.0,NaN,NaN,Maneuvering,0.0
2663185,1006946,319646000,660707,PARAFFIN,2001-10-01,Yacht,Other,2023-09-30 02:20:41+00:00,2023-09-30,103.786190,...,PPT Maneouvering Zone,2984.0,14.0,195.0,230.0,300.0,NaN,NaN,Maneuvering,41.0
2658468,1006946,319646000,660707,PARAFFIN,2001-10-01,Yacht,Other,2023-09-30 02:25:08+00:00,2023-09-30,103.802060,...,NaN,2984.0,14.0,195.0,230.0,300.0,NaN,NaN,Transit,308.0
2658467,1006946,319646000,660707,PARAFFIN,2001-10-01,Yacht,Other,2023-09-30 02:25:40+00:00,2023-09-30,103.803820,...,NaN,2984.0,14.0,195.0,230.0,300.0,NaN,NaN,Transit,340.0
2658466,1006946,319646000,660707,PARAFFIN,2001-10-01,Yacht,Other,2023-09-30 02:30:08+00:00,2023-09-30,103.818245,...,NaN,2984.0,14.0,195.0,230.0,300.0,NaN,NaN,Transit,608.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2679458,9997440,563190700,976919,MAJESTIC GLORY,2023-04-01,Passenger Ship,Passenger/Cruise,2023-09-30 10:00:55+00:00,2023-09-30,103.811226,...,NaN,3300.0,28.0,195.0,230.0,300.0,NaN,NaN,Transit,43.0
2687878,9997440,563190700,976919,MAJESTIC GLORY,2023-04-01,Passenger Ship,Passenger/Cruise,2023-09-30 12:20:00+00:00,2023-09-30,103.812390,...,NaN,3300.0,28.0,195.0,230.0,300.0,NaN,NaN,Transit,8388.0
2687877,9997440,563190700,976919,MAJESTIC GLORY,2023-04-01,Passenger Ship,Passenger/Cruise,2023-09-30 12:20:03+00:00,2023-09-30,103.812390,...,NaN,3300.0,28.0,195.0,230.0,300.0,NaN,NaN,Transit,8391.0
2687876,9997440,563190700,976919,MAJESTIC GLORY,2023-04-01,Passenger Ship,Passenger/Cruise,2023-09-30 13:30:01+00:00,2023-09-30,103.808710,...,NaN,3300.0,28.0,195.0,230.0,300.0,NaN,NaN,Transit,12589.0


In [8]:
# Step 3: Determine the vessel type


vessel_types = ['Bulk Carrier', 'Container Ship (Fully Cellular)', 'General Cargo Ship','General Cargo Ship (with Ro-Ro facility)', 
                'LPG/Chemical Tanker', 'Passenger/Cruise',
                'Products Tanker', 'Refrigerated Cargo Ship', 'Ro-Ro Cargo Ship', 'Chemical Tanker']

# Create a mask for rows where 'container types' column is in the specified vessel types
mask = ship_movements['vessel_type'].isin(vessel_types)

# Create a new DataFrame with only the rows that satisfy the condition
ship_movements = ship_movements[mask]
ship_movements

,imo,mmsi,vessel_id,vessel_name,date_of_build,vessel_type,group,timestamp,date_only,lon,...,maneuvering_zone,p,vref,sfc_me,sfc_ae,sfc_ab,ael,abl,operating_mode,time_diff
1867922,7305722,416567000,740022,SHENG NO. 66,1973-06-01,Refrigerated Cargo Ship,Reefer,2023-08-05 01:25:55+00:00,2023-08-05,103.824486,...,NaN,3972.0,14.75,225.0,230.0,300.0,NaN,NaN,Transit,931.0
1867923,7305722,416567000,740022,SHENG NO. 66,1973-06-01,Refrigerated Cargo Ship,Reefer,2023-08-05 01:30:03+00:00,2023-08-05,103.820915,...,NaN,3972.0,14.75,225.0,230.0,300.0,74.0,-24.0,Transit,1179.0
1851642,7305722,416567000,740022,SHENG NO. 66,1973-06-01,Refrigerated Cargo Ship,Reefer,2023-08-05 01:30:05+00:00,2023-08-05,103.820915,...,NaN,3972.0,14.75,225.0,230.0,300.0,74.0,-24.0,Transit,1181.0
1851643,7305722,416567000,740022,SHENG NO. 66,1973-06-01,Refrigerated Cargo Ship,Reefer,2023-08-05 01:35:03+00:00,2023-08-05,103.822050,...,NaN,3972.0,14.75,225.0,230.0,300.0,74.0,-24.0,Transit,1479.0
1851644,7305722,416567000,740022,SHENG NO. 66,1973-06-01,Refrigerated Cargo Ship,Reefer,2023-08-05 01:35:15+00:00,2023-08-05,103.822070,...,NaN,3972.0,14.75,225.0,230.0,300.0,74.0,-24.0,Transit,1491.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2658503,9982988,352002938,974117,CRUX TWIN,2023-09-01,General Cargo Ship,General Cargo,2023-09-30 09:10:16+00:00,2023-09-30,103.764824,...,NaN,3030.0,13.80,195.0,230.0,300.0,374.0,117.0,Transit,8749.0
2658502,9982988,352002938,974117,CRUX TWIN,2023-09-01,General Cargo Ship,General Cargo,2023-09-30 09:15:04+00:00,2023-09-30,103.755830,...,NaN,3030.0,13.80,195.0,230.0,300.0,374.0,117.0,Transit,9037.0
2658501,9982988,352002938,974117,CRUX TWIN,2023-09-01,General Cargo Ship,General Cargo,2023-09-30 09:15:34+00:00,2023-09-30,103.754950,...,NaN,3030.0,13.80,195.0,230.0,300.0,374.0,117.0,Transit,9067.0
2658500,9982988,352002938,974117,CRUX TWIN,2023-09-01,General Cargo Ship,General Cargo,2023-09-30 09:20:14+00:00,2023-09-30,103.747475,...,NaN,3030.0,13.80,195.0,230.0,300.0,374.0,117.0,Transit,9347.0


In [9]:
# Step 4: Determine Emissions Tier
 
ship_movements['date_of_build'] = pd.to_datetime(ship_movements['date_of_build'], errors='coerce')

# Extract the year from the build date
ship_movements['Model_Year'] = ship_movements['date_of_build'].dt.year

# Define a function to determine the emissions tier based on the Model Year
def determine_emissions_tier(model_year):
    if model_year > 2015:
        return 3
    elif model_year <= 2015 and model_year >= 2011:
        return 2
    elif model_year <= 2010 and model_year >= 2000:
        return 1
    else:
        return 0

# Apply the function to create a new 'Emissions Tier' column
ship_movements['emission_tier'] = ship_movements['Model_Year'].apply(determine_emissions_tier)

ship_movements

C:\Users\User\AppData\Local\Temp\ipykernel_3836\1714501236.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ship_movements['date_of_build'] = pd.to_datetime(ship_movements['date_of_build'], errors='coerce')
C:\Users\User\AppData\Local\Temp\ipykernel_3836\1714501236.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ship_movements['Model_Year'] = ship_movements['date_of_build'].dt.year
C:\Users\User\AppData\Local\Temp\ipykernel_3836\1714501236.py:20: SettingWithCopyWarning: 
A value is trying to be set o

,imo,mmsi,vessel_id,vessel_name,date_of_build,vessel_type,group,timestamp,date_only,lon,...,vref,sfc_me,sfc_ae,sfc_ab,ael,abl,operating_mode,time_diff,Model_Year,emission_tier
1867922,7305722,416567000,740022,SHENG NO. 66,1973-06-01,Refrigerated Cargo Ship,Reefer,2023-08-05 01:25:55+00:00,2023-08-05,103.824486,...,14.75,225.0,230.0,300.0,NaN,NaN,Transit,931.0,1973,0
1867923,7305722,416567000,740022,SHENG NO. 66,1973-06-01,Refrigerated Cargo Ship,Reefer,2023-08-05 01:30:03+00:00,2023-08-05,103.820915,...,14.75,225.0,230.0,300.0,74.0,-24.0,Transit,1179.0,1973,0
1851642,7305722,416567000,740022,SHENG NO. 66,1973-06-01,Refrigerated Cargo Ship,Reefer,2023-08-05 01:30:05+00:00,2023-08-05,103.820915,...,14.75,225.0,230.0,300.0,74.0,-24.0,Transit,1181.0,1973,0
1851643,7305722,416567000,740022,SHENG NO. 66,1973-06-01,Refrigerated Cargo Ship,Reefer,2023-08-05 01:35:03+00:00,2023-08-05,103.822050,...,14.75,225.0,230.0,300.0,74.0,-24.0,Transit,1479.0,1973,0
1851644,7305722,416567000,740022,SHENG NO. 66,1973-06-01,Refrigerated Cargo Ship,Reefer,2023-08-05 01:35:15+00:00,2023-08-05,103.822070,...,14.75,225.0,230.0,300.0,74.0,-24.0,Transit,1491.0,1973,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2658503,9982988,352002938,974117,CRUX TWIN,2023-09-01,General Cargo Ship,General Cargo,2023-09-30 09:10:16+00:00,2023-09-30,103.764824,...,13.80,195.0,230.0,300.0,374.0,117.0,Transit,8749.0,2023,3
2658502,9982988,352002938,974117,CRUX TWIN,2023-09-01,General Cargo Ship,General Cargo,2023-09-30 09:15:04+00:00,2023-09-30,103.755830,...,13.80,195.0,230.0,300.0,374.0,117.0,Transit,9037.0,2023,3
2658501,9982988,352002938,974117,CRUX TWIN,2023-09-01,General Cargo Ship,General Cargo,2023-09-30 09:15:34+00:00,2023-09-30,103.754950,...,13.80,195.0,230.0,300.0,374.0,117.0,Transit,9067.0,2023,3
2658500,9982988,352002938,974117,CRUX TWIN,2023-09-01,General Cargo Ship,General Cargo,2023-09-30 09:20:14+00:00,2023-09-30,103.747475,...,13.80,195.0,230.0,300.0,374.0,117.0,Transit,9347.0,2023,3


In [10]:
# step 5:
def determine_fuel_category(ship_movements):
    ship_movements['fuel_category'] = "Global 2020"
    return ship_movements
    
ship_movements = determine_fuel_category(ship_movements)
ship_movements

C:\Users\User\AppData\Local\Temp\ipykernel_3836\3433820232.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ship_movements['fuel_category'] = "Global 2020"


,imo,mmsi,vessel_id,vessel_name,date_of_build,vessel_type,group,timestamp,date_only,lon,...,vref,sfc_me,sfc_ae,sfc_ab,ael,abl,operating_mode,time_diff,Model_Year,emission_tier
1867922,7305722,416567000,740022,SHENG NO. 66,1973-06-01,Refrigerated Cargo Ship,Reefer,2023-08-05 01:25:55+00:00,2023-08-05,103.824486,...,14.75,225.0,230.0,300.0,NaN,NaN,Transit,931.0,1973,0
1867923,7305722,416567000,740022,SHENG NO. 66,1973-06-01,Refrigerated Cargo Ship,Reefer,2023-08-05 01:30:03+00:00,2023-08-05,103.820915,...,14.75,225.0,230.0,300.0,74.0,-24.0,Transit,1179.0,1973,0
1851642,7305722,416567000,740022,SHENG NO. 66,1973-06-01,Refrigerated Cargo Ship,Reefer,2023-08-05 01:30:05+00:00,2023-08-05,103.820915,...,14.75,225.0,230.0,300.0,74.0,-24.0,Transit,1181.0,1973,0
1851643,7305722,416567000,740022,SHENG NO. 66,1973-06-01,Refrigerated Cargo Ship,Reefer,2023-08-05 01:35:03+00:00,2023-08-05,103.822050,...,14.75,225.0,230.0,300.0,74.0,-24.0,Transit,1479.0,1973,0
1851644,7305722,416567000,740022,SHENG NO. 66,1973-06-01,Refrigerated Cargo Ship,Reefer,2023-08-05 01:35:15+00:00,2023-08-05,103.822070,...,14.75,225.0,230.0,300.0,74.0,-24.0,Transit,1491.0,1973,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2658503,9982988,352002938,974117,CRUX TWIN,2023-09-01,General Cargo Ship,General Cargo,2023-09-30 09:10:16+00:00,2023-09-30,103.764824,...,13.80,195.0,230.0,300.0,374.0,117.0,Transit,8749.0,2023,3
2658502,9982988,352002938,974117,CRUX TWIN,2023-09-01,General Cargo Ship,General Cargo,2023-09-30 09:15:04+00:00,2023-09-30,103.755830,...,13.80,195.0,230.0,300.0,374.0,117.0,Transit,9037.0,2023,3
2658501,9982988,352002938,974117,CRUX TWIN,2023-09-01,General Cargo Ship,General Cargo,2023-09-30 09:15:34+00:00,2023-09-30,103.754950,...,13.80,195.0,230.0,300.0,374.0,117.0,Transit,9067.0,2023,3
2658500,9982988,352002938,974117,CRUX TWIN,2023-09-01,General Cargo Ship,General Cargo,2023-09-30 09:20:14+00:00,2023-09-30,103.747475,...,13.80,195.0,230.0,300.0,374.0,117.0,Transit,9347.0,2023,3


In [11]:
# Step 6: Retrieve specific ship details for all vessels that are within the port boundary

from shapely.geometry import Point

def iswithinportboundary(lon, lat):
    point=Point(lon, lat)
    return point.within(port_boundary.unary_union)

ship_movements_within_port_boundary = ship_movements.copy()
ship_movements_within_port_boundary['Point'] = ship_movements_within_port_boundary.apply(lambda row: Point(row['lon'], row['lat']), axis=1)
gdf_ship_movements_within_port_boundary = gpd.GeoDataFrame(ship_movements_within_port_boundary, geometry='Point')

# Use the 'within' method to create a mask for rows within the port boundary
mask_within_boundary = gdf_ship_movements_within_port_boundary.geometry.within(port_boundary.unary_union)

# Filter the DataFrame based on the mask
ship_movements_within_port_boundary = ship_movements_within_port_boundary[mask_within_boundary]

# Drop the temporary 'Point' column if you no longer need it
ship_movements_within_port_boundary = ship_movements_within_port_boundary.drop(columns=['Point'])
ship_movements_within_port_boundary

,imo,mmsi,vessel_id,vessel_name,date_of_build,vessel_type,group,timestamp,date_only,lon,...,vref,sfc_me,sfc_ae,sfc_ab,ael,abl,operating_mode,time_diff,Model_Year,emission_tier
1867922,7305722,416567000,740022,SHENG NO. 66,1973-06-01,Refrigerated Cargo Ship,Reefer,2023-08-05 01:25:55+00:00,2023-08-05,103.824486,...,14.75,225.0,230.0,300.0,NaN,NaN,Transit,931.0,1973,0
1867923,7305722,416567000,740022,SHENG NO. 66,1973-06-01,Refrigerated Cargo Ship,Reefer,2023-08-05 01:30:03+00:00,2023-08-05,103.820915,...,14.75,225.0,230.0,300.0,74.0,-24.0,Transit,1179.0,1973,0
1851642,7305722,416567000,740022,SHENG NO. 66,1973-06-01,Refrigerated Cargo Ship,Reefer,2023-08-05 01:30:05+00:00,2023-08-05,103.820915,...,14.75,225.0,230.0,300.0,74.0,-24.0,Transit,1181.0,1973,0
1851643,7305722,416567000,740022,SHENG NO. 66,1973-06-01,Refrigerated Cargo Ship,Reefer,2023-08-05 01:35:03+00:00,2023-08-05,103.822050,...,14.75,225.0,230.0,300.0,74.0,-24.0,Transit,1479.0,1973,0
1851644,7305722,416567000,740022,SHENG NO. 66,1973-06-01,Refrigerated Cargo Ship,Reefer,2023-08-05 01:35:15+00:00,2023-08-05,103.822070,...,14.75,225.0,230.0,300.0,74.0,-24.0,Transit,1491.0,1973,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2658503,9982988,352002938,974117,CRUX TWIN,2023-09-01,General Cargo Ship,General Cargo,2023-09-30 09:10:16+00:00,2023-09-30,103.764824,...,13.80,195.0,230.0,300.0,374.0,117.0,Transit,8749.0,2023,3
2658502,9982988,352002938,974117,CRUX TWIN,2023-09-01,General Cargo Ship,General Cargo,2023-09-30 09:15:04+00:00,2023-09-30,103.755830,...,13.80,195.0,230.0,300.0,374.0,117.0,Transit,9037.0,2023,3
2658501,9982988,352002938,974117,CRUX TWIN,2023-09-01,General Cargo Ship,General Cargo,2023-09-30 09:15:34+00:00,2023-09-30,103.754950,...,13.80,195.0,230.0,300.0,374.0,117.0,Transit,9067.0,2023,3
2658500,9982988,352002938,974117,CRUX TWIN,2023-09-01,General Cargo Ship,General Cargo,2023-09-30 09:20:14+00:00,2023-09-30,103.747475,...,13.80,195.0,230.0,300.0,374.0,117.0,Transit,9347.0,2023,3


In [12]:
# Step 7b: Calculate maximum speed
ship_movements["MaxSpeed"] = ship_movements["vref"]*1.066 
ship_movements

C:\Users\User\AppData\Local\Temp\ipykernel_3836\2978337227.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ship_movements["MaxSpeed"] = ship_movements["vref"]*1.066


,imo,mmsi,vessel_id,vessel_name,date_of_build,vessel_type,group,timestamp,date_only,lon,...,sfc_me,sfc_ae,sfc_ab,ael,abl,operating_mode,time_diff,Model_Year,emission_tier,MaxSpeed
1867922,7305722,416567000,740022,SHENG NO. 66,1973-06-01,Refrigerated Cargo Ship,Reefer,2023-08-05 01:25:55+00:00,2023-08-05,103.824486,...,225.0,230.0,300.0,NaN,NaN,Transit,931.0,1973,0,15.7235
1867923,7305722,416567000,740022,SHENG NO. 66,1973-06-01,Refrigerated Cargo Ship,Reefer,2023-08-05 01:30:03+00:00,2023-08-05,103.820915,...,225.0,230.0,300.0,74.0,-24.0,Transit,1179.0,1973,0,15.7235
1851642,7305722,416567000,740022,SHENG NO. 66,1973-06-01,Refrigerated Cargo Ship,Reefer,2023-08-05 01:30:05+00:00,2023-08-05,103.820915,...,225.0,230.0,300.0,74.0,-24.0,Transit,1181.0,1973,0,15.7235
1851643,7305722,416567000,740022,SHENG NO. 66,1973-06-01,Refrigerated Cargo Ship,Reefer,2023-08-05 01:35:03+00:00,2023-08-05,103.822050,...,225.0,230.0,300.0,74.0,-24.0,Transit,1479.0,1973,0,15.7235
1851644,7305722,416567000,740022,SHENG NO. 66,1973-06-01,Refrigerated Cargo Ship,Reefer,2023-08-05 01:35:15+00:00,2023-08-05,103.822070,...,225.0,230.0,300.0,74.0,-24.0,Transit,1491.0,1973,0,15.7235
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2658503,9982988,352002938,974117,CRUX TWIN,2023-09-01,General Cargo Ship,General Cargo,2023-09-30 09:10:16+00:00,2023-09-30,103.764824,...,195.0,230.0,300.0,374.0,117.0,Transit,8749.0,2023,3,14.7108
2658502,9982988,352002938,974117,CRUX TWIN,2023-09-01,General Cargo Ship,General Cargo,2023-09-30 09:15:04+00:00,2023-09-30,103.755830,...,195.0,230.0,300.0,374.0,117.0,Transit,9037.0,2023,3,14.7108
2658501,9982988,352002938,974117,CRUX TWIN,2023-09-01,General Cargo Ship,General Cargo,2023-09-30 09:15:34+00:00,2023-09-30,103.754950,...,195.0,230.0,300.0,374.0,117.0,Transit,9067.0,2023,3,14.7108
2658500,9982988,352002938,974117,CRUX TWIN,2023-09-01,General Cargo Ship,General Cargo,2023-09-30 09:20:14+00:00,2023-09-30,103.747475,...,195.0,230.0,300.0,374.0,117.0,Transit,9347.0,2023,3,14.7108


In [13]:
# Step 7d (i): Calculate Engine Load Factor (LF) for propulsion engines
ship_movements["LF"] = (ship_movements["speed"]/ship_movements["MaxSpeed"])**3  
ship_movements

C:\Users\User\AppData\Local\Temp\ipykernel_3836\3372227684.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ship_movements["LF"] = (ship_movements["speed"]/ship_movements["MaxSpeed"])**3


,imo,mmsi,vessel_id,vessel_name,date_of_build,vessel_type,group,timestamp,date_only,lon,...,sfc_ae,sfc_ab,ael,abl,operating_mode,time_diff,Model_Year,emission_tier,MaxSpeed,LF
1867922,7305722,416567000,740022,SHENG NO. 66,1973-06-01,Refrigerated Cargo Ship,Reefer,2023-08-05 01:25:55+00:00,2023-08-05,103.824486,...,230.0,300.0,NaN,NaN,Transit,931.0,1973,0,15.7235,NaN
1867923,7305722,416567000,740022,SHENG NO. 66,1973-06-01,Refrigerated Cargo Ship,Reefer,2023-08-05 01:30:03+00:00,2023-08-05,103.820915,...,230.0,300.0,74.0,-24.0,Transit,1179.0,1973,0,15.7235,0.000257
1851642,7305722,416567000,740022,SHENG NO. 66,1973-06-01,Refrigerated Cargo Ship,Reefer,2023-08-05 01:30:05+00:00,2023-08-05,103.820915,...,230.0,300.0,74.0,-24.0,Transit,1181.0,1973,0,15.7235,0.000257
1851643,7305722,416567000,740022,SHENG NO. 66,1973-06-01,Refrigerated Cargo Ship,Reefer,2023-08-05 01:35:03+00:00,2023-08-05,103.822050,...,230.0,300.0,74.0,-24.0,Transit,1479.0,1973,0,15.7235,0.000000
1851644,7305722,416567000,740022,SHENG NO. 66,1973-06-01,Refrigerated Cargo Ship,Reefer,2023-08-05 01:35:15+00:00,2023-08-05,103.822070,...,230.0,300.0,74.0,-24.0,Transit,1491.0,1973,0,15.7235,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2658503,9982988,352002938,974117,CRUX TWIN,2023-09-01,General Cargo Ship,General Cargo,2023-09-30 09:10:16+00:00,2023-09-30,103.764824,...,230.0,300.0,374.0,117.0,Transit,8749.0,2023,3,14.7108,0.160828
2658502,9982988,352002938,974117,CRUX TWIN,2023-09-01,General Cargo Ship,General Cargo,2023-09-30 09:15:04+00:00,2023-09-30,103.755830,...,230.0,300.0,374.0,117.0,Transit,9037.0,2023,3,14.7108,0.160828
2658501,9982988,352002938,974117,CRUX TWIN,2023-09-01,General Cargo Ship,General Cargo,2023-09-30 09:15:34+00:00,2023-09-30,103.754950,...,230.0,300.0,374.0,117.0,Transit,9067.0,2023,3,14.7108,0.160828
2658500,9982988,352002938,974117,CRUX TWIN,2023-09-01,General Cargo Ship,General Cargo,2023-09-30 09:20:14+00:00,2023-09-30,103.747475,...,230.0,300.0,374.0,117.0,Transit,9347.0,2023,3,14.7108,0.067849


In [14]:
llaf = pd.read_csv(r'C:\Users\User\Downloads\llaf_table.csv')
percentages = llaf['Load']
llaf['LFrounded']= [float(value.strip('%')) / 100 for value in percentages]
llaf

,Load,NOx,HC,CO,PM,SO2,CO2,N2O,CH4,LFrounded
0,2%,4.63,21.18,9.68,7.29,3.36,3.28,4.63,21.18,0.02
1,3%,2.92,11.68,6.46,4.33,2.49,2.44,2.92,11.68,0.03
2,4%,2.21,7.71,4.86,3.09,2.05,2.01,2.21,7.71,0.04
3,5%,1.83,5.61,3.89,2.44,1.79,1.76,1.83,5.61,0.05
4,6%,1.60,4.35,3.25,2.04,1.61,1.59,1.60,4.35,0.06
5,7%,1.45,3.52,2.79,1.79,1.49,1.47,1.45,3.52,0.07
6,8%,1.35,2.95,2.45,1.61,1.39,1.38,1.35,2.95,0.08
7,9%,1.27,2.52,2.18,1.48,1.32,1.31,1.27,2.52,0.09
8,10%,1.22,2.20,1.96,1.38,1.26,1.25,1.22,2.20,0.10
9,11%,1.17,1.96,1.79,1.30,1.21,1.21,1.17,1.96,0.11


In [15]:
ship_movements['LFrounded'] = ship_movements['LF'].round(decimals = 2)
# replace to 1 if > 0.2
ship_movements['LFrounded'] = ship_movements['LFrounded'].apply(lambda x: 0.2 if x > 0.2 else x)
ship_movements = pd.merge(ship_movements, llaf, on='LFrounded')
ship_movements

C:\Users\User\AppData\Local\Temp\ipykernel_3836\2714035488.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ship_movements['LFrounded'] = ship_movements['LF'].round(decimals = 2)
C:\Users\User\AppData\Local\Temp\ipykernel_3836\2714035488.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ship_movements['LFrounded'] = ship_movements['LFrounded'].apply(lambda x: 0.2 if x > 0.2 else x)


,imo,mmsi,vessel_id,vessel_name,date_of_build,vessel_type,group,timestamp,date_only,lon,...,LFrounded,Load,NOx,HC,CO,PM,SO2,CO2,N2O,CH4
0,7323401,416602000,778868,SHUN TIAN FA NO. 168,1973-12-01,Refrigerated Cargo Ship,Reefer,2023-08-04 00:30:43+00:00,2023-08-04,103.800316,...,0.19,19%,1.01,1.05,1.05,1.02,1.01,1.01,1.01,1.05
1,7323401,416602000,778868,SHUN TIAN FA NO. 168,1973-12-01,Refrigerated Cargo Ship,Reefer,2023-09-26 06:25:04+00:00,2023-09-26,103.745630,...,0.19,19%,1.01,1.05,1.05,1.02,1.01,1.01,1.01,1.05
2,8649620,312693000,764556,HUA CHEN 26,2006-12-01,Bulk Carrier,Bulk Carrier,2023-08-01 16:09:02+00:00,2023-08-01,103.787670,...,0.19,19%,1.01,1.05,1.05,1.02,1.01,1.01,1.01,1.05
3,8649620,312693000,764556,HUA CHEN 26,2006-12-01,Bulk Carrier,Bulk Carrier,2023-08-01 16:15:49+00:00,2023-08-01,103.801125,...,0.19,19%,1.01,1.05,1.05,1.02,1.01,1.01,1.01,1.05
4,8649620,312693000,764556,HUA CHEN 26,2006-12-01,Bulk Carrier,Bulk Carrier,2023-08-04 01:37:52+00:00,2023-08-04,103.787670,...,0.19,19%,1.01,1.05,1.05,1.02,1.01,1.01,1.01,1.05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62164,9952086,413395130,965152,YONG AO,2022-12-01,Products Tanker,Crude & Products Tanker,2023-09-04 02:30:03+00:00,2023-09-04,103.790280,...,0.17,17%,1.03,1.18,1.17,1.06,1.05,1.04,1.03,1.18
62165,9952098,413513510,965153,YONG BO,2023-04-01,Products Tanker,Crude & Products Tanker,2023-08-12 00:05:40+00:00,2023-08-12,103.776726,...,0.17,17%,1.03,1.18,1.17,1.06,1.05,1.04,1.03,1.18
62166,9952098,413513510,965153,YONG BO,2023-04-01,Products Tanker,Crude & Products Tanker,2023-08-15 23:55:03+00:00,2023-08-15,103.793340,...,0.17,17%,1.03,1.18,1.17,1.06,1.05,1.04,1.03,1.18
62167,9952098,413513510,965153,YONG BO,2023-04-01,Products Tanker,Crude & Products Tanker,2023-08-16 00:10:00+00:00,2023-08-16,103.765370,...,0.17,17%,1.03,1.18,1.17,1.06,1.05,1.04,1.03,1.18


In [16]:
#Nox
propulsion = pd.read_csv(r'C:\Users\User\Downloads\propulsion_engine_emission_factors.csv')
boiler = pd.read_csv(r'C:\Users\User\Downloads\boiler_engine_emission_factors.csv')
aux = pd.read_csv(r'C:\Users\User\Downloads\auxiliary_engine_emission_factors.csv')

propulsion_subset = propulsion[(propulsion["fuel"] == "Global 2020")]

# Merge ship_movements and propulsion_subset on specified columns
merged_df = pd.merge(ship_movements, propulsion_subset, 
                     on=["emission_tier", "engine_type"],
                     how="left")

# Update "propNOx" column in ship_movements based on the conditions
ship_movements["propNOx"] = merged_df["nox"]

# Display the updated ship_movements DataFrame
print(ship_movements)
import numpy as np

# Assuming aux DataFrame is already loaded

# Use numpy.where to assign values based on the condition
aux["engine_type"] = np.where(aux["engine_size"] == "< 800 kW", "SSD", "MSD")
aux_subset = aux[(aux["fuel"] == "Global 2020")]
# Merge ship_movements and propulsion_subset on specified columns
merged_df1 = pd.merge(ship_movements, aux_subset, 
                     on=["emission_tier", "engine_type"],
                     how="left")

# Update "propNOx" column in ship_movements based on the conditions
ship_movements["auxNOx"] = merged_df["nox"]
ship_movements["boilerNOx"] = 2

#pm10
ship_movements['mainPM10'] = 0.23 + ship_movements['sfc_me'] * 7 * 0.02247 * (0.005-0.0024)
ship_movements['auxPM10'] = 0.23 + ship_movements['sfc_ae'] * 7 * 0.02247 * (0.005-0.0024)
ship_movements['boilerPM10'] = 0.352686

#pm2.5
ship_movements['mainPM2.5'] = 0.92 * ship_movements['mainPM10']
ship_movements['auxPM2.5'] = 0.92 * ship_movements['auxPM10']
ship_movements['boilerPM2.5'] = 0.92 * ship_movements['boilerPM10']


#SOx – Refer to vessel movements table.
ship_movements['mainSOx'] = ship_movements['sfc_me']* 2 * 0.97753 * 0.005
ship_movements['auxSOx'] = ship_movements['sfc_ae']* 2 * 0.97753 * 0.005
ship_movements['boilerSOx'] = ship_movements['sfc_ab']* 2 * 0.97753 * 0.005

# CO2 - Refer to vessel movements table
ship_movements['mainCO2'] = ship_movements['sfc_me']* 0.867 * 3.6675
ship_movements['auxCO2'] = ship_movements['sfc_ae']* 0.867 * 3.6675
ship_movements['boilerCO2'] = ship_movements['sfc_ab']* 0.867 * 3.6675

#Ch4
ship_movements['mainCH4'] = 0.01
ship_movements['auxCH4'] = 0.01
ship_movements['boilerCH4'] = 0.002

#N2O
ship_movements['mainN2O'] = merged_df["n2_O"]
ship_movements['auxN2O'] = 0.036
ship_movements['boilerN2O'] = 0.049



           imo       mmsi  vessel_id           vessel_name date_of_build  \
0      7323401  416602000     778868  SHUN TIAN FA NO. 168    1973-12-01   
1      7323401  416602000     778868  SHUN TIAN FA NO. 168    1973-12-01   
2      8649620  312693000     764556           HUA CHEN 26    2006-12-01   
3      8649620  312693000     764556           HUA CHEN 26    2006-12-01   
4      8649620  312693000     764556           HUA CHEN 26    2006-12-01   
...        ...        ...        ...                   ...           ...   
62164  9952086  413395130     965152               YONG AO    2022-12-01   
62165  9952098  413513510     965153               YONG BO    2023-04-01   
62166  9952098  413513510     965153               YONG BO    2023-04-01   
62167  9952098  413513510     965153               YONG BO    2023-04-01   
62168  9952098  413513510     965153               YONG BO    2023-04-01   

                   vessel_type                    group  \
0      Refrigerated Cargo Sh

In [17]:
time_diff=(ship_movements['time_diff'] / 3600)
#Propulsion engine emissions
# NOx	HC	CO	PM	SO2	CO2	N2O	CH4
# P x LF x A x EF x LLAF
ship_movements['PropulsionNOx']=ship_movements['p']*ship_movements['LF']*time_diff*ship_movements['propNOx'] * ship_movements['NOx']
ship_movements['PropulsionPM2.5']=ship_movements['p']*ship_movements['LF']*time_diff*ship_movements['mainPM2.5']*ship_movements['PM']
ship_movements['PropulsionPM10']=ship_movements['p']*ship_movements['LF']*time_diff*ship_movements['mainPM10']*ship_movements['PM']
ship_movements['PropulsionSO2']=ship_movements['p']*ship_movements['LF']*time_diff*ship_movements['mainSOx']*ship_movements['SO2']
ship_movements['PropulsionCO2']=ship_movements['p']*ship_movements['LF']*time_diff*ship_movements['mainCO2']*ship_movements['CO2']
ship_movements['PropulsionCH4']=ship_movements['p']*ship_movements['LF']*time_diff*ship_movements['mainCH4']*ship_movements['CH4']
ship_movements['PropulsionN2O']=ship_movements['p']*ship_movements['LF']*time_diff*ship_movements['mainN2O']*ship_movements['N2O']

#Auxiliary engine emissions
# LA X A x EF
ship_movements['AuxiliaryNOx']=ship_movements['ael']*time_diff*ship_movements['auxNOx']
ship_movements['AuxiliaryPM2.5']=ship_movements['ael']*time_diff*ship_movements['auxPM2.5']
ship_movements['AuxiliaryPM10']=ship_movements['ael']*time_diff*ship_movements['auxPM10']
ship_movements['AuxiliarySO2']=ship_movements['ael']*time_diff*ship_movements['auxSOx']
ship_movements['AuxiliaryCO2']=ship_movements['ael']*time_diff*ship_movements['auxCO2']
ship_movements['AuxiliaryCH4']=ship_movements['ael']*time_diff*ship_movements['auxCH4']
ship_movements['AuxiliaryN2O']=ship_movements['ael']*time_diff*ship_movements['auxN2O']

#Auxiliary boiler  engine emissions
# E = LB X A x EF
ship_movements['AuxiliaryboilerNOx']=ship_movements['abl']*time_diff*ship_movements['boilerNOx']
ship_movements['AuxiliaryboilerPM2.5']=ship_movements['abl']*time_diff*ship_movements['boilerPM2.5']
ship_movements['AuxiliaryboilerPM10']=ship_movements['abl']*time_diff*ship_movements['boilerPM10']
ship_movements['AuxiliaryboilerSO2']=ship_movements['abl']*time_diff*ship_movements['boilerSOx']
ship_movements['AuxiliaryboilerCO2']=ship_movements['abl']*time_diff*ship_movements['boilerCO2']
ship_movements['AuxiliaryboilerCH4']=ship_movements['abl']*time_diff*ship_movements['boilerCH4']
ship_movements['AuxiliaryboilerN2O']=ship_movements['abl']*time_diff*ship_movements['boilerN2O']

In [18]:
ship_movements["operating_mode"].items

<bound method Series.items of 0            Transit
1            Transit
2        Maneuvering
3            Transit
4        Maneuvering
            ...     
62164    Maneuvering
62165    Maneuvering
62166        Transit
62167        Transit
62168        Transit
Name: operating_mode, Length: 62169, dtype: object>

In [19]:
# Overall electrical load
def total_electrical_load(ship_movements):
    alongside_ships = ship_movements[ship_movements['operating_mode'] == "Hotel"]
    alongside_ships['duration_hours'] = alongside_ships['time_diff'].dt.total_seconds() / 3600
    alongside_ships['time_weighted_ael'] = alongside_ships['ael'] * alongside_ships['duration_hours']

    # Calculate the total alongside duration in hours
    total_alongside_duration = alongside_ships['duration_hours'].sum()

    # Calculate the time-weighted average electrical load (Tael)
    result = alongside_ships['time_weighted_ael'].sum() / total_alongside_duration
    print(result)
    return result

ship_movements1 = pd.read_csv(r'C:\Users\User\Downloads\vessel_movements_PPT.csv')
ship_movements1 = log_events(ship_movements1)
ship_movements1['operating_mode'] = ship_movements1.apply(determine_operating_mode, axis=1)
ship_movements1=total_electrical_load(ship_movements1)


NameError: name 'log_events' is not defined

In [ ]:
#total co2 emissions in port boundary
def iswithinportboundary(lon, lat):
    point=Point(lon, lat)
    return point.within(port_boundary.unary_union)

ship_movements_within_port_boundary = ship_movements.copy()
ship_movements_within_port_boundary['Point'] = ship_movements_within_port_boundary.apply(lambda row: Point(row['lon'], row['lat']), axis=1)
gdf_ship_movements_within_port_boundary = gpd.GeoDataFrame(ship_movements_within_port_boundary, geometry='Point')

# Use the 'within' method to create a mask for rows within the port boundary
mask_within_boundary = gdf_ship_movements_within_port_boundary.geometry.within(port_boundary.unary_union)

# Filter the DataFrame based on the mask
ship_movements_within_port_boundary = ship_movements_within_port_boundary[mask_within_boundary]

# Drop the temporary 'Point' column if you no longer need it
ship_movements_within_port_boundary = ship_movements_within_port_boundary.drop(columns=['Point'])

total_co2_emissions_within_port = ship_movements_within_port_boundary['AuxiliaryboilerCO2'].sum()+ship_movements_within_port_boundary['AuxiliaryCO2'].sum() + ship_movements_within_port_boundary['PropulsionCO2'].sum()
print(total_co2_emissions_within_port)

In [ ]:
# CO2 Emissions when the vessels are in anchor,manoeuvring, alongside mode
ship_movements_copy = ship_movements.copy()


# Filter the DataFrame based on the selected operating modes
filtered_ship_movements_copy_transit = ship_movements_copy[ship_movements_copy['operating_mode']=='Transit']
filtered_ship_movements_copy_anchor = ship_movements_copy[ship_movements_copy['operating_mode']=='Anchorage']
filtered_ship_movements_copy_manoeuvring= ship_movements_copy[ship_movements_copy['operating_mode']=='Maneuvering']
filtered_ship_movements_copy_alongside = ship_movements_copy[ship_movements_copy['operating_mode']=='Hotel']

# Calculate the total CO2 emissions for the selected operating modes
total_co2_emissions_transit = filtered_ship_movements_copy_transit['AuxiliaryboilerCO2'].sum()+filtered_ship_movements_copy_transit['AuxiliaryCO2'].sum() + filtered_ship_movements_copy_transit['PropulsionCO2'].sum()
total_co2_emissions_anchor = filtered_ship_movements_copy_anchor['AuxiliaryboilerCO2'].sum()+filtered_ship_movements_copy_anchor['AuxiliaryCO2'].sum() + filtered_ship_movements_copy_anchor['PropulsionCO2'].sum()
total_co2_emissions_manoeuvring = filtered_ship_movements_copy_manoeuvring['AuxiliaryboilerCO2'].sum()+filtered_ship_movements_copy_manoeuvring['AuxiliaryCO2'].sum() + filtered_ship_movements_copy_manoeuvring['PropulsionCO2'].sum()
total_co2_emissions_alongside = filtered_ship_movements_copy_alongside['AuxiliaryboilerCO2'].sum()+filtered_ship_movements_copy_alongside['AuxiliaryCO2'].sum() + filtered_ship_movements_copy_alongside['PropulsionCO2'].sum()

# Print or use the total CO2 emissions for the selected modes as needed
print("Total CO2 emissions for transit mode:", total_co2_emissions_transit)
print("Total CO2 emissions for anchor mode:", total_co2_emissions_anchor)
print("Total CO2 emissions for manoeuvring mode:", total_co2_emissions_manoeuvring)
print("Total CO2 emissions for alongside mode:", total_co2_emissions_alongside)

In [ ]:
#calculate day night load variation

import seaborn as sns
import matplotlib.pyplot as plt
def calculate_day_night_load_variation(ship_movements):
    # Filter 'Alongside' operation mode
    ship_movements['timestamp'] = pd.to_datetime(ship_movements['timestamp'])

    day_start = pd.to_datetime('09:00:00').time()
    day_end = pd.to_datetime('17:00:00').time()
    night_start = pd.to_datetime('17:00:00').time()
    night_end = pd.to_datetime('09:00:00').time()
    
    day_mask = (ship_movements['timestamp'].dt.time >= day_start) & (ship_movements['timestamp'].dt.time <= day_end)
    night_mask = (ship_movements['timestamp'].dt.time >= night_start) | (ship_movements['timestamp'].dt.time <= night_end)
    
    DayEL = ship_movements[day_mask]
    NightEL = ship_movements[night_mask]
    print("Day: "+DayEL.astype(str))
    print("NIght: "+NightEL.astype(str))

    plt.figure(figsize=(12, 6))

    sns.countplot(x=DayEL['timestamp'].dt.hour, color='blue', label='Day')
    sns.countplot(x=NightEL['timestamp'].dt.hour, color='orange', label='Night')

    plt.title('Daily Demand Variation')
    plt.xlabel('Hour of the Day')
    plt.ylabel('Number of Operations')
    plt.legend()
    plt.show()

    plt.figure(figsize=(14, 6))

    ship_movements['Month'] = ship_movements['timestamp'].dt.month
    sns.countplot(x='Month', data=ship_movements, palette='viridis')

    plt.title('Monthly Demand Variation')
    plt.xlabel('Month')
    plt.ylabel('Number of Operations')
    plt.show()
    total_electrical_load(DayEL)
    total_electrical_load(NightEL)
    
calculate_day_night_load_variation(ship_movements)



In [ ]:
# Number of unique vessels that visited the port

unique_vessels = ship_movements['vessel_id'].nunique()
print(unique_vessels)